In [1]:
from omop_etl.datastore import DataStore
from omop_etl.load import Loader
import pandas as pd

import os
os.chdir('Z:/Covid19/Covid19_OMOP/new_pipeline/')
print(os.getcwd())

Z:\Covid19\Covid19_OMOP\new_pipeline


In [2]:
def flatten(lst):
    for el in lst:
        if isinstance(el, list):  
            # recurse
            yield from flatten(el)
        else:
            # generate
            yield el

In [3]:
omop = DataStore('config.yml')

In [4]:
with omop.connection() as con:
    dev = pd.read_sql('''
    SELECT distinct [respiratory_device]
      FROM [DWS_OMOP].[stage].[MEASUREMENT_Res_Device]
    ''', con)

In [5]:
# dev.respiratory_device.apply(lambda s: s.split(';')).to_list()

In [6]:
# with omop.connection() as con:
#     vmode = pd.read_sql('''
#     SELECT distinct adult_vent_mode
#       FROM [DWS_OMOP].[stage].[MEASUREMENT_Res_Vent]
#     ''', con)

In [7]:
set(list(flatten(dev.respiratory_device.apply(lambda s: s.split(';')).to_list())))

{'Aerosol mask',
 'BVM (Bag Valve Mask)',
 'BiPAP',
 'Blow-by',
 'CPAP',
 'Cricothyrotomy',
 'ETT',
 'Face tent',
 'High Flow via Trach',
 'High flow nasal cannula',
 'IGel',
 'King Tube',
 'LMA',
 'Nasal cannula',
 'Non-rebreather mask',
 'Oscillator',
 'Other (Comment)',
 'Oxyhood',
 'Oxyimiser',
 'Partial rebreather mask',
 'Positive Airway Pressure Device',
 'Room Air / (None)',
 'Simple mask',
 'T-piece',
 'Trach mask',
 'Tracheostomy',
 'Transtracheal catheter',
 'Ventilator',
 'Venturi mask'}

In [62]:
from omop_etl.utils import search
import re

In [123]:
with omop.connection() as con:
    icd = pd.read_sql('''
        select distinct b.DIAG_CD_LVL2_DESC as [Desc], b.ICD_TYPE, count(distinct DIAG_CD_DECML) N
        from dbo.condition_occurrence a
        join dws_prod.dbo.ALL_ICD_DIAGNOSIS_CODES b
        on a.condition_source_value = b.DIAG_CD_DECML
        group by b.DIAG_CD_LVL2_DESC, b.ICD_TYPE
        order by b.DIAG_CD_LVL2_DESC--count(*) desc
    ''', con)

In [124]:
icd = icd[['ICD_TYPE', 'Desc', 'N']]

In [125]:
# icd

In [126]:
def as_sentence(s):
    splitted = s.split(None, 1) if search('[0-9]', s) else s
    if len(splitted) == 2:
        icd,des = splitted
        return icd + ': ' + des.capitalize()
    else:
        des = splitted[0]
        return splitted.capitalize()

In [127]:
icd['Desc'] = icd.apply(lambda row: re.sub('\(.*.\)', '', as_sentence(row.Desc)) + f' ({str(row.N)})', 1)

In [128]:
icd.to_csv('z:/OMOP/omop_etl/notebooks/icd_codes.csv', index=False)

In [129]:
icd.apply(lambda row: row.Desc, 1)

0        001-009.99: Intestinal infectious diseases (43)
1                          010-018.99: Tuberculosis (32)
2            020-027.99: Zoonotic bacterial diseases (9)
3              030-041.99: Other bacterial diseases (70)
4      042-042.99: Human immunodeficiency virus [hiv]...
                             ...                        
561              Z66-Z66: Do not resuscitate status  (1)
562                            Z67-Z67: Blood type  (10)
563                 Z68-Z68: Body mass index [bmi]  (30)
564    Z69-Z76: Persons encountering health services ...
565    Z77-Z99: Persons with potential health hazards...
Length: 566, dtype: object